<a href="https://colab.research.google.com/github/horv1tz/neural-network-rinh/blob/main/NNRINH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Обучение нейросети

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, accuracy_score
import numpy as np
import joblib

# Загрузка данных
data = pd.read_csv('credit_score_ds.csv', sep='|')

# Предобработка данных
numeric_features = ['age', 'annual_income', 'monthly_inhand_salary', 'num_bank_accounts', 'num_credit_card', 'num_of_loan', 'num_credit_inquiries', 'credit_history_age', 'amount_invested_monthly', 'monthly_balance']
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_features = ['occupation', 'payment_behaviour']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Разделение данных
X = data.drop('credit_score', axis=1)
y = data['credit_score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Создание модели
model = Sequential([
    Dense(128, activation='relu', input_shape=[X_train.shape[1]]),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Компиляция модели
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Обучение модели
history = model.fit(X_train, y_train, batch_size=128, epochs=500, validation_split=0.0001, verbose=1)

# Предсказания модели на тестовом наборе
y_pred = model.predict(X_test).ravel()  # ravel() превращает матрицу предсказаний в вектор

# Оценка точности
accuracy = accuracy_score(y_test, y_pred.round())
print(f'Точность модели: {accuracy}')

# Вычисление AUC
auc = roc_auc_score(y_test, y_pred)
print(f'AUC модели: {auc}')

# Сохранение модели
model.save('credit_score_model.h5')
print("Модель сохранена как credit_score_model.h5")

# Сохранение предобработчика
joblib.dump(preprocessor, 'preprocessor.pkl')
print("Предобработчик сохранён как preprocessor.pkl")

Epoch 1/500
625/625 [==============================] - 2s 3ms/step - loss: 0.4834 - accuracy: 0.7693 - val_loss: 0.2078 - val_accuracy: 1.0000
Epoch 2/500
625/625 [==============================] - 2s 3ms/step - loss: 0.4668 - accuracy: 0.7793 - val_loss: 0.2053 - val_accuracy: 1.0000
Epoch 3/500
625/625 [==============================] - 2s 4ms/step - loss: 0.4611 - accuracy: 0.7822 - val_loss: 0.2388 - val_accuracy: 1.0000
Epoch 4/500
625/625 [==============================] - 2s 3ms/step - loss: 0.4564 - accuracy: 0.7853 - val_loss: 0.2390 - val_accuracy: 1.0000
Epoch 5/500
625/625 [==============================] - 1s 2ms/step - loss: 0.4523 - accuracy: 0.7888 - val_loss: 0.2855 - val_accuracy: 0.8750
Epoch 6/500
625/625 [==============================] - 2s 2ms/step - loss: 0.4480 - accuracy: 0.7912 - val_loss: 0.2921 - val_accuracy: 0.7500
Epoch 7/500
625/625 [==============================] - 1s 2ms/step - loss: 0.4427 - accuracy: 0.7935 - val_loss: 0.3423 - val_accuracy: 0.7500

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#Пример использования

In [ ]:
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.compose import ColumnTransformer

def use_model(data):
  df_to_predict = pd.DataFrame(data)

  # Предполагаем, что `preprocessor` - это объект ColumnTransformer, который был обучен на обучающих данных
  # Здесь должен быть ваш код для создания и обучения `preprocessor`

  # Загрузка модели
  model = load_model('credit_score_model.h5')

  # Предобработка данных для предсказания
  X_to_predict = preprocessor.transform(df_to_predict)

  # Делаем предсказание
  predicted_credit_score = model.predict(X_to_predict)
  return predicted_credit_score

# Создаем тестовый DataFrame
data = {
    'month': [8],
    'age': [30],
    'occupation': ['Media_Manager'],
    'annual_income': [34000],
    'monthly_inhand_salary': [2600],
    'num_bank_accounts': [8],
    'num_credit_card': [7],
    'num_of_loan': [3],
    'num_credit_inquiries': [9],
    'credit_history_age': [182],
    'amount_invested_monthly': [29],
    'payment_behaviour': ['High_spent_Medium_value_payments'],
    'monthly_balance': [411]
}

# Выводим предсказанные значения
print(use_model(data))


1/1 [==============================] - 0s 49ms/step
[[0.5898668]]


#Проверка нейросети на датасете

In [13]:
import pandas as pd
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import load_model

# Загрузка модели
model = load_model('credit_score_model.h5')

# Загрузка датасета
data = pd.read_csv('credit_score_ds.csv', sep='|')

# Определение числовых и категориальных признаков
numeric_features = ['age', 'annual_income', 'monthly_inhand_salary', 'num_bank_accounts', 'num_credit_card', 'num_of_loan', 'num_credit_inquiries', 'credit_history_age', 'amount_invested_monthly', 'monthly_balance']
categorical_features = ['occupation', 'payment_behaviour']

# Создание обработчиков для предобработки
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Предобработка данных
X = preprocessor.fit_transform(data.drop('credit_score', axis=1))
y_true = data['credit_score'].values

# Выполнение предсказания
y_pred_probs = model.predict(X).flatten()  # Получаем вероятности принадлежности к классу

# Перевод вероятностей в бинарные значения для расчета точности
y_pred = (y_pred_probs > 0.5).astype(int)

# Расчет точности
accuracy = accuracy_score(y_true, y_pred)
print(f"Точность модели: {accuracy * 100:.2f}%")

# Расчет AUC
auc = roc_auc_score(y_true, y_pred_probs)
print(f"AUC модели: {auc:.2f}")

3125/3125 [==============================] - 6s 2ms/step
Точность модели: 91.16%
AUC модели: 0.97
